In [1]:
!git clone https://github.com/phuc22062004/FinalCuda.git

Cloning into 'FinalCuda'...
remote: Enumerating objects: 393, done.
remote: Counting objects: 100% (359/359), done.
remote: Compressing objects: 100% (196/196), done.
remote: Total 393 (delta 193), reused 302 (delta 139), pack-reused 34 (from 1)
Receiving objects: 100% (393/393), 326.09 MiB | 15.71 MiB/s, done.
Resolving deltas: 100% (199/199), done.
Updating files: 100% (25/25), done.


In [15]:
%cd /content/FinalCuda/thundersvm

# 1. Clean up the previous broken build
!rm -rf build
!mkdir build
%cd build

# 2. Detect your GPU (T4, P100, etc.)
import os
# Run nvidia-smi to get the compute capability (e.g., "7.5")
compute_cap = !nvidia-smi --query-gpu=compute_cap --format=csv,noheader
if not compute_cap:
    # Fallback to T4 (sm_75) if detection fails
    arch_flag = "-arch=sm_75"
    print("Could not detect GPU. Defaulting to T4 (sm_75).")
else:
    # Convert "7.5" -> "sm_75"
    ver = compute_cap[0].replace('.', '')
    arch_flag = f"-arch=sm_{ver}"
    print(f"Detected GPU: {compute_cap[0]}. Using flag: {arch_flag}")

# 3. Configure CMake using the OLD-STYLE flag (Critical Step)
# We use -DCUDA_NVCC_FLAGS instead of -DCMAKE_CUDA_ARCHITECTURES
command = f'cmake .. -DCMAKE_BUILD_TYPE=Release -DCUDA_NVCC_FLAGS="{arch_flag}"'
get_ipython().system(command)

# 4. Build again
!make -j4

# 5. Go back to root
%cd ../..

/content/FinalCuda/thundersvm
/content/FinalCuda/thundersvm/build
Detected GPU: 7.5. Using flag: -arch=sm_75
CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.10 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value.  Or, use the <min>...<max> syntax
  to tell CMake that the project requires at least <min> but has been updated
  to work with policies introduced by <max> or earlier.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Te

In [3]:
%cd /content/FinalCuda
!git submodule update --init --recursive
%cd thundersvm/
!mkdir build
%cd build
!cmake .. -DCMAKE_BUILD_TYPE=Release -DCMAKE_CUDA_ARCHITECTURES=75
!make -j4
%cd ../..

/content/FinalCuda
Submodule 'thundersvm' (https://github.com/Xtra-Computing/thundersvm.git) registered for path 'thundersvm'
Cloning into '/content/FinalCuda/thundersvm'...
Submodule path 'thundersvm': checked out '5c6a056ac7f474b085d5415c81c5d48a1419642a'
Submodule 'eigen' (https://github.com/eigenteam/eigen-git-mirror) registered for path 'thundersvm/eigen'
Submodule 'src/test/googletest' (https://github.com/google/googletest.git) registered for path 'thundersvm/src/test/googletest'
Cloning into '/content/FinalCuda/thundersvm/eigen'...
Cloning into '/content/FinalCuda/thundersvm/src/test/googletest'...
Submodule path 'thundersvm/eigen': checked out 'dde02fceedfc1ba09d4d4f71a2b5dafcfcb85491'
Submodule path 'thundersvm/src/test/googletest': checked out 'a325ad2db5deb623eab740527e559b81c0f39d65'
/content/FinalCuda/thundersvm
/content/FinalCuda/thundersvm/build
CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.10 will be removed from 

## Building script

In [6]:
!chmod +x scripts/*

In [8]:
!./scripts/build_cuda.sh

Detecting GPU architecture...
Detected GPU: sm_75. Using flag: -arch=sm_75
Building CUDA versions with -arch=sm_75...
Building basic version...
Building opt_v1 (Memory Optimized)...
Building opt_v2 (Speed Optimized)...
CUDA binaries created successfully in: /content/FinalCuda/build_cuda


In [9]:
!./scripts/build_svm.sh

Detecting GPU architecture...
Detected GPU: sm_75. Using flag: -arch=sm_75
Building SVM Feature Extraction Tools...
Building CPU feature extractor...
Building CUDA feature extractor (Basic)...
Building CUDA OPT_V1 feature extractor...
Building CUDA OPT_V2 feature extractor...

SVM feature extractors built successfully!
  - extract_features_cpu
  - extract_features_cuda (basic)
  - extract_features_cuda_opt_v1
  - extract_features_cuda_opt_v2


## GPU Basic

In [11]:
!./build_cuda/autoencoder_cuda_basic ./cifar-10-binary/cifar-10-batches-bin/ weights/autoencoder/gpu_basic.bin 1 64 0.001 5000

=== CUDA BASIC (Phase 2) ===
CIFAR dir: ./cifar-10-binary/cifar-10-batches-bin/
Weights:   weights/autoencoder/gpu_basic.bin
Epochs:    1
Batch:     64
LR:        0.001
Max train: 5000

Loaded train images: 50000
Using 5000 images (debug limit)

Epoch 1/1 - shuffling...
  Batch 1 loss: 0.110099
  Batch 10 loss: 0.0346273
  Batch 20 loss: 0.0295723
  Batch 30 loss: 0.023421
  Batch 40 loss: 0.0226826
  Batch 50 loss: 0.0223575
  Batch 60 loss: 0.0206973
  Batch 70 loss: 0.0191148
Epoch avg loss: 0.0273784 | time: 32s

Total training time: 33s
Saving weights to weights/autoencoder/gpu_basic.bin
Done.


In [12]:
!./build_svm/extract_features_cuda ./cifar-10-binary/cifar-10-batches-bin/ \
  weights/autoencoder/gpu_basic.bin weights/thundersvm/train_features_basic.libsvm weights/thundersvm/test_features_basic.libsvm

=== CUDA Feature Extraction for SVM (With Z-Score Scaling) ===
CIFAR-10 dir: ./cifar-10-binary/cifar-10-batches-bin/
Weights:      weights/autoencoder/gpu_basic.bin
Output train: weights/thundersvm/train_features_basic.libsvm
Output test:  weights/thundersvm/test_features_basic.libsvm
Scaler:       scaler_z.bin
Fast mode:    NO (binary + libsvm)

Loaded train images: 50000
Loaded test images:  10000

Loaded weights from: weights/autoencoder/gpu_basic.bin
Loaded autoencoder weights

PASS 1: Extracting train features to cache (chunked + pinned memory)...
  Processed 0/50000
  Completed 50000/50000
  PASS 1 breakdown:
    - Pure GPU extraction:  49s
    - Cache write I/O:      1s
  PASS 1 total time:        50s

Computing statistics (mean/std) from cache...
  Statistics computed (mean/std for 8192 dims)
  Stats computation time: 1s
  Scaler saved to: scaler_z.bin

PASS 2: Scaling and writing binary features...
  Binary features written to:
    weights/thundersvm/train_features_basic.libsv

In [23]:
!./thundersvm/build/bin/thundersvm-train -s 0 -t 2 -c 10 -u 0 weights/thundersvm/train_features_basic.libsvm weights/svm/model_basic.txt

2025-12-14 10:38:29,916 INFO dataset.cpp:21 : loading dataset from file "weights/thundersvm/train_features_basic.libsvm"
2025-12-14 10:41:36,800 INFO dataset.cpp:99 : #instances = 50000, #features = 8192
2025-12-14 10:41:36,800 INFO thundersvm-train.cpp:34 : training C-SVC
2025-12-14 10:41:36,800 INFO thundersvm-train.cpp:35 : C = 10
2025-12-14 10:41:36,800 WARNING thundersvm-train.cpp:76 : using default gamma=0.00012207
2025-12-14 10:41:39,110 INFO svc.cpp:20 : #classes = 10
2025-12-14 10:41:42,928 INFO svmmodel.cpp:445 : total memory size is 0.915863 max mem size is 8
2025-12-14 10:41:42,928 INFO svmmodel.cpp:446 : free mem is 7.08414
2025-12-14 10:41:42,932 INFO svmmodel.cpp:449 : working set size = 1024
2025-12-14 10:41:42,934 INFO csmosolver.cpp:48 : training start
2025-12-14 10:41:43,242 INFO csmosolver.cpp:104 : global iter = 0, total local iter = 458, diff = 2
2025-12-14 10:41:44,254 INFO csmosolver.cpp:113 : global iter = 32, total local iter = 15837, diff = 0.000988581
2025-1

In [ ]:
# 1. Create a 10GB swap file on the disk
!fallocate -l 10G /swapfile

# 2. Fix permissions
!chmod 600 /swapfile

# 3. Mark it as swap space
!mkswap /swapfile

# 4. Enable the swap
!swapon /swapfile

# 5. Verify it worked (Look for the 'Swap' row)
!free -h

In [27]:
!./thundersvm/build/bin/thundersvm-train -s 0 -t 2 -c 10 -u 0 weights/thundersvm/train_features_basic.libsvm weights/svm/model_basic.txt

2025-12-14 11:03:44,284 INFO dataset.cpp:21 : loading dataset from file "weights/thundersvm/train_features_basic.libsvm"
2025-12-14 11:06:51,346 INFO dataset.cpp:99 : #instances = 50000, #features = 8192
2025-12-14 11:06:51,346 INFO thundersvm-train.cpp:34 : training C-SVC
2025-12-14 11:06:51,346 INFO thundersvm-train.cpp:35 : C = 0.01
2025-12-14 11:06:51,346 WARNING thundersvm-train.cpp:76 : using default gamma=0.00012207
2025-12-14 11:06:53,864 INFO svc.cpp:20 : #classes = 10
2025-12-14 11:06:58,508 INFO svmmodel.cpp:445 : total memory size is 0.915863 max mem size is 8
2025-12-14 11:06:58,508 INFO svmmodel.cpp:446 : free mem is 7.08414
2025-12-14 11:06:58,512 INFO svmmodel.cpp:449 : working set size = 1024
2025-12-14 11:06:58,513 INFO csmosolver.cpp:48 : training start
2025-12-14 11:06:58,839 INFO csmosolver.cpp:104 : global iter = 0, total local iter = 512, diff = 2
2025-12-14 11:06:59,369 INFO csmosolver.cpp:113 : global iter = 17, total local iter = 4032, diff = 0.000921683
2025-

In [21]:
!./thundersvm/build/bin/thundersvm-predict weights/thundersvm/test_features_basic.libsvm weights/svm/model_basic.txt results/prediction_gpu.txt

2025-12-14 10:34:44,719 FATAL [default] Check failed: [feature == "svm_type"] 
2025-12-14 10:34:44,719 WARNING [default] Aborting application. Reason: Fatal log at [/content/FinalCuda/thundersvm/src/thundersvm/thundersvm-predict.cpp:31]


In [24]:
!head weights/svm/model_basic.txt
!ls -lh weights/svm/model_basic.txt

-rw-r--r-- 1 root root 0 Dec 14 10:45 weights/svm/model_basic.txt


In [14]:
!nvidia-smi --query-gpu=compute_cap --format=csv,noheader

7.5
